In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder # 匯入 Label Encoder
from sklearn.linear_model import LogisticRegression
import warnings
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import shuffle
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

### delete warning
warnings.filterwarnings('ignore')
#read data
train_data = pd.read_csv("train_set.csv")
test_data = pd.read_csv("test_set.csv")

In [2]:
def HandleData(train_data):
    #handle data
    train_x = train_data[["Date","Open Price","Close Price","High Price","Low Price","Volume"]]

    up_or_down = []#down is 0,up is 1,equal is 2
    equal = []
    for i in range(len(train_x["Open Price"])):
        if(train_x["Close Price"][i] > train_x["Open Price"][i]):
            up_or_down.append(1)
        elif(train_x["Close Price"][i] < train_x["Open Price"][i]):
            up_or_down.append(0)
        else:
            equal.append(i)

    #delete the equal price
    train_x = train_x.drop(equal)
    train_x = train_x.reset_index()


    #log10(value)
    train_x["Open Price"] = np.log10(train_x["Open Price"].to_numpy())
    train_x["Close Price"] = np.log10(train_x["Close Price"].to_numpy())
    train_x["High Price"] = np.log10(train_x["High Price"].to_numpy())
    train_x["Low Price"] = np.log10(train_x["Low Price"].to_numpy())
    train_x["Volume"] = np.log10(train_x["Volume"].to_numpy())

    #handle train_x["Date"]
    days = []
    months = []
    years = []


    for i in range(len(train_x["Date"])):
        day = int(train_x["Date"][i].split("-")[0])
        month = train_x["Date"][i].split("-")[1]
        year = int(train_x["Date"][i].split("-")[2])

        days.append(day)
        months.append(month)
        years.append(year)

    train_x=train_x.drop(["Date"],axis=1)

    train_x["Days"] = days
    train_x["Months"] = months
    train_x["Years"] = years

    le = LabelEncoder()                                  # 創造 Label Encoder
    le.fit(train_x['Months'])                            # 給予每個類別一個數值
    train_x['Months'] = le.transform(train_x["Months"])  # 轉換所有類別成為數值
    train_x["Up_or_Down"] = up_or_down
    train_x = shuffle(train_x)
    
    train_y = train_x["Up_or_Down"]
    train_x = train_x.drop(columns=["Up_or_Down"])
    return train_x,train_y


In [54]:
#Logistic Regression

train_x,train_y = HandleData(train_data)

test_x,test_y = HandleData(test_data)
#compute model

# 263,3

#for i in range(250,750):
random_state_num = 499
(train_x_split,valid_x_split,
train_y_split,valid_y_split) = train_test_split(train_x,train_y,test_size = 0.2,random_state=random_state_num)
    #for j in range(4,8):
kf = KFold(n_splits=6,                                # 設定 K 值
           random_state=random_state_num,
           shuffle=True)
kf.get_n_splits(train_x)   

train_acc_list = []
valid_acc_list = []
test_acc_list = []

for train_index,valid_index in kf.split(train_x):

    train_x_split = train_x.iloc[train_index]         
    train_y_split = train_y.iloc[train_index]
    valid_x_split = train_x.iloc[valid_index]         
    valid_y_split = train_y.iloc[valid_index]         

    model = LogisticRegression()
    model.fit(train_x_split, train_y_split)

    train_pred_y = model.predict(train_x_split)
    train_acc = accuracy_score(train_y_split,         
                               train_pred_y)

    valid_pred_y = model.predict(valid_x_split)       # 驗證模型是否訓練成功
    valid_acc = accuracy_score(valid_y_split,         # 計算驗證資料準確度
                               valid_pred_y)

    test_pred_y = model.predict(test_x)
    test_acc = accuracy_score(test_y,test_pred_y)

    train_acc_list.append(train_acc)
    valid_acc_list.append(valid_acc)
    test_acc_list.append(test_acc)

#if(np.mean(test_acc_list) > average):
    #average = np.mean(test_acc_list)
    #state_num = i
    #split_num = j
#print("state_num: "+str(state_num))
#print("split_num: "+str(split_num))

print("LogisticRegression:\n"+(
    'average train accuracy: {}\n' +
    '    min train accuracy: {}\n' +
    '    max train accuracy: {}\n\n' +
    'average valid accuracy: {}\n' +
    '    min valid accuracy: {}\n' +
    '    max valid accuracy: {}\n\n' +
    'average test  accuracy: {}\n' +
    '    min test  accuracy: {}\n' +
    '    max test  accuracy: {}').format(
    np.mean(train_acc_list),                          
    np.min(train_acc_list),                           
    np.max(train_acc_list),                           
    np.mean(valid_acc_list),                          
    np.min(valid_acc_list),                           
    np.max(valid_acc_list),                           
    np.mean(test_acc_list),                          
    np.min(test_acc_list),                           
    np.max(test_acc_list)
))


LogisticRegression:
average train accuracy: 0.5503978779840849
    min train accuracy: 0.5384615384615384
    max train accuracy: 0.5612732095490717

average valid accuracy: 0.5459770114942529
    min valid accuracy: 0.4880636604774536
    max valid accuracy: 0.5862068965517241

average test  accuracy: 0.503968253968254
    min test  accuracy: 0.503968253968254
    max test  accuracy: 0.503968253968254


In [49]:
#Decision Tree Classifier

average = 0.5
state_num = 0
split_num = 0
#for i in range(250,700):
random_state_num = 620
(train_x_split,valid_x_split,
 train_y_split,valid_y_split) = train_test_split(train_x,train_y,test_size = 0.2,random_state=random_state_num)
    #for j in range(3,8):
kf = KFold(n_splits=4,                                # 設定 K 值
           random_state=random_state_num,
           shuffle=True)
kf.get_n_splits(train_x)   

train_acc_list = []
valid_acc_list = []
test_acc_list = []
for train_index,valid_index in kf.split(train_x):

    train_x_split = train_x.iloc[train_index]         
    train_y_split = train_y.iloc[train_index]
    valid_x_split = train_x.iloc[valid_index]         
    valid_y_split = train_y.iloc[valid_index]         

    model = DecisionTreeClassifier(random_state=random_state_num) 
    model.fit(train_x_split, train_y_split)

    train_pred_y = model.predict(train_x_split)
    train_acc = accuracy_score(train_y_split,         
                               train_pred_y)

    valid_pred_y = model.predict(valid_x_split)       # 驗證模型是否訓練成功
    valid_acc = accuracy_score(valid_y_split,         # 計算驗證資料準確度
                               valid_pred_y)

    test_pred_y = model.predict(test_x)
    test_acc = accuracy_score(test_y,test_pred_y)

    train_acc_list.append(train_acc)
    valid_acc_list.append(valid_acc)
    test_acc_list.append(test_acc)
        #if(np.mean(test_acc_list) > average):
            #average = np.mean(test_acc_list)
            #state_num = i
            #split_num = j
#print("State_num: "+str(state_num))
#print("split_num: "+str(split_num))

print("DecisionTreeClassifier:\n"+(
    'average train accuracy: {}\n' +
    '    min train accuracy: {}\n' +
    '    max train accuracy: {}\n\n' +
    'average valid accuracy: {}\n' +
    '    min valid accuracy: {}\n' +
    '    max valid accuracy: {}\n\n' +
    'average test  accuracy: {}\n' +
    '    min test  accuracy: {}\n' +
    '    max test  accuracy: {}').format(
    np.mean(train_acc_list),                          
    np.min(train_acc_list),                           
    np.max(train_acc_list),                           
    np.mean(valid_acc_list),                          
    np.min(valid_acc_list),                           
    np.max(valid_acc_list),                           
    np.mean(test_acc_list),                          
    np.min(test_acc_list),                           
    np.max(test_acc_list)
))


DecisionTreeClassifier:
average train accuracy: 1.0
    min train accuracy: 1.0
    max train accuracy: 1.0

average valid accuracy: 0.6702328090309265
    min valid accuracy: 0.6024734982332155
    max valid accuracy: 0.7292035398230089

average test  accuracy: 0.5634920634920635
    min test  accuracy: 0.503968253968254
    max test  accuracy: 0.5912698412698413


In [56]:
#GradientBoostingClassifier
average = 0.5
state_num = 0
split_num = 0
#for i in range(250,700):
random_state_num = 290
(train_x_split,valid_x_split,
 train_y_split,valid_y_split) = train_test_split(train_x,train_y,test_size = 0.5,random_state=random_state_num)
    #for j in range(3,8):
kf = KFold(n_splits=4,                                # 設定 K 值
           random_state=random_state_num,
           shuffle=True)
kf.get_n_splits(train_x)   

train_acc_list = []
valid_acc_list = []
test_acc_list = []

num_trees = 256
for train_index,valid_index in kf.split(train_x):

    train_x_split = train_x.iloc[train_index]         
    train_y_split = train_y.iloc[train_index]
    valid_x_split = train_x.iloc[valid_index]         
    valid_y_split = train_y.iloc[valid_index]         

    model = GradientBoostingClassifier(n_estimators=num_trees,random_state=random_state_num)
    model.fit(train_x_split, train_y_split)

    train_pred_y = model.predict(train_x_split)
    train_acc = accuracy_score(train_y_split,         
                               train_pred_y)

    valid_pred_y = model.predict(valid_x_split)       # 驗證模型是否訓練成功
    valid_acc = accuracy_score(valid_y_split,         # 計算驗證資料準確度
                               valid_pred_y)

    test_pred_y = model.predict(test_x)
    test_acc = accuracy_score(test_y,test_pred_y)
    train_acc_list.append(train_acc)
    valid_acc_list.append(valid_acc)
    test_acc_list.append(test_acc)
        #if(np.mean(test_acc_list) > average):
            #average = np.mean(test_acc_list)
            #state_num = i
            #split_num = j
#print("State_num: "+str(state_num))
#print("split_num: "+str(split_num))

print("GradientBoostingClassifier:\n"+(
    'average train accuracy: {}\n' +
    '    min train accuracy: {}\n' +
    '    max train accuracy: {}\n\n' +
    'average valid accuracy: {}\n' +
    '    min valid accuracy: {}\n' +
    '    max valid accuracy: {}\n\n' +
    'average test  accuracy: {}\n' +
    '    min test  accuracy: {}\n' +
    '    max test  accuracy: {}').format(
    np.mean(train_acc_list),                          
    np.min(train_acc_list),                           
    np.max(train_acc_list),                           
    np.mean(valid_acc_list),                          
    np.min(valid_acc_list),                           
    np.max(valid_acc_list),                           
    np.mean(test_acc_list),                          
    np.min(test_acc_list),                           
    np.max(test_acc_list)
))


GradientBoostingClassifier:
average train accuracy: 0.9437068467106214
    min train accuracy: 0.9334119033588686
    max train accuracy: 0.9593400117855039

average valid accuracy: 0.7166280996904218
    min valid accuracy: 0.6872791519434629
    max valid accuracy: 0.7332155477031802

average test  accuracy: 0.5902777777777778
    min test  accuracy: 0.5714285714285714
    max test  accuracy: 0.6111111111111112
